In [ ]:
from src.sampling.images import plot_frame_count_distributions

samples_root_dir = "data/samples"

plot_frame_count_distributions(samples_root_dir)

## Sota model training 

In [ ]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11m-cls")
sota.initialize_model()
sota.train_model(optimizer="AdamW", lr0=0.0005)

#metrics = model.val(data="data/img/techniques/val")

In [ ]:
from sota.model import SOTA

sota = SOTA("data", "yolo11n-cls")
sota.initialize_model()
sota.train_model()

In [ ]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11m-balance-50-155", dataset_name="techniques_balanced")
sota.initialize_model("yolo11m-cls")
sota.train_model()

In [ ]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11n-balance-50-155", dataset_name="techniques_balanced")
sota.initialize_model("yolo11n-cls")
#sota.train_model()
sota.test_model()

In [ ]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11n-cls-full-balanced")
sota.execute_train_runs(model="yolo11n-cls", runs=2, epochs=10, balanced=True)

In [1]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11m-cls-full-balanced")
sota.execute_train_runs(model="yolo11m-cls", runs=2, epochs=10, balanced=True)

/mnt/c/Projects/climbing-technique-detector/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


starting run #0
loading a fresh model 'yolo11m-cls'


wandb: Currently logged in as: tom-decoutere (tom-decoutere-open-universiteit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


New https://pypi.org/project/ultralytics/8.3.166 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.78 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA TITAN RTX, 24576MiB)
engine/trainer: task=classify, mode=train, model=yolo11m-cls.pt, data=data/img/techniques, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=data/runs/sota/yolo11m-cls-full-balanced, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=

E0000 00:00:1752570832.808133   30454 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752570833.090799   30454 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

               classes   top1_acc   top5_acc: 100%|██████████| 26/26 [00:06<00:00,  3.86it/s]


                   all      0.826      0.993


Generating Visualizations for batch-1/1: 100%|██████████| 1/1 [00:00<00:00, 197.70it/s]

Speed: 0.4ms preprocess, 2.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to data/runs/sota/yolo11m-cls-full-balanced/train2


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f5b4843c170>
curves: []
curves_results: []
fitness: 0.909203976392746
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8258706331253052, 'metrics/accuracy_top5': 0.9925373196601868, 'fitness': 0.909203976392746}
save_dir: PosixPath('data/runs/sota/yolo11m-cls-full-balanced/train2')
speed: {'preprocess': 0.4248630112009443, 'inference': 2.190324718901423, 'loss': 0.005786312185455923, 'postprocess': 0.003232557222628514}
task: 'classify'
top1: 0.8258706331253052
top5: 0.9925373196601868


In [4]:
f"{sota.model.overrides["model"]}".split('.')[0]

'yolo11m-cls'

In [ ]:
from ultralytics import YOLO
from wandb import finish
from wandb.integration.ultralytics import add_wandb_callback

model = YOLO("yolo11m-cls")
add_wandb_callback(model, enable_model_checkpointing=True)
model.train(data="coco8.yaml", epochs=1, imgsz=640)
finish()


New https://pypi.org/project/ultralytics/8.3.166 available 😃 Update with 'pip install -U ultralytics'


SyntaxError: '[31m[1mepoch[0m' is not a valid YOLO argument. Similar arguments are i.e. ['epochs=100'].

    Arguments received: ['yolo', '--f=/run/user/1000/jupyter/runtime/kernel-v302c5efccc7a2d89e09c7f7cdb6852ffa2adb1fbe.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of frozenset({'obb', 'segment', 'pose', 'detect', 'classify'})
                MODE (required) is one of frozenset({'benchmark', 'predict', 'train', 'track', 'export', 'val'})
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco8.yaml model=yolo11n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmentation model at image size 320:
        yolo

## HPE DNN model training

In [ ]:
from src.hpe_dnn.model import HpeDnn

hpednn = HpeDnn("data", "arch1")
hpednn.initialize_model()
hpednn.train_model()

In [ ]:
from src.hpe_dnn.model import HpeDnn

hpednn_aug = HpeDnn("data", "arch1_augmented")
hpednn_aug.initialize_model()
hpednn_aug.train_model(augment=True)

In [ ]:
%load_ext tensorboard

from src.hpe_dnn.model import HpeDnn

hpednn = HpeDnn("data", "arch1_balanced", "techniques_balanced")
hpednn.initialize_model()
hpednn.train_model()

In [ ]:
from src.hpe_dnn.model import HpeDnn

hpednn = HpeDnn("data", "arch1_balanced_augmented", "techniques_balanced")
hpednn.initialize_model()
hpednn.train_model(augment=True)

In [ ]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch2_balanced", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH2)
hpednn.train_model()

In [ ]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch3_balanced", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH3)
hpednn.train_model()

In [ ]:
from src.hpe_dnn.model import HpeDnn

hpednn = HpeDnn("data", "arch1_full_balanced")
hpednn.execute_train_runs(runs=2, epochs=10, augment=True, balanced=True)

In [ ]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch1_balanced_not_norm", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH1, normalize=False)
hpednn.train_model()

In [ ]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch1_balanced_dr_0.3", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH1, dropout_rate=0.3)
hpednn.train_model()

In [ ]:
%tensorboard --logdir data/runs/hpe_dnn/arch1_balanced/train1/logs/train

In [ ]:
from src.hpe_dnn.model import read_data

df_path = "data/df/techniques/train.pkl"
train = read_data(df_path)